En esta libreta vamos a experimentar con el análisis de tópicos a partir de un corpus en español muy interesante para realizar análisis de tópicos: Contiene transcripciones de las informes diarios sobre coronavirus en México desde el inicio de febrero del 2020 organizados por archivos csv, sin embargo para este trabajo vamos a considerar únicamente las participaciones realizadas por el Dr. Hugo López-Gatell Ramírez.

El corpus se encuentra [en este proyecto de github](https://github.com/NOSTRODATA/informe_diario_sobre_coronavirus_en_mexico) y pueden encontrar más información a cerca de [NOSTRODATA](https://www.nostrodata.com) en su página de internet.


## 1. Obtención del *corpus*

Para obtener el *corpus* es necesario clonar el proyecto original, se usa el comando:

```
git clone https://github.com/NOSTRODATA/informe_diario_sobre_coronavirus_en_mexico.git
```

Una vez clonado, es necesario extraer la información de los archivos `csv`. La generación del *corpus* sin normalizar ya la dejo desarollada.

En mi caso realicé una separación de los archivos csv para contar solamente con las participaciones que relaizo el Dr. Hugo López-Gatell Ramirez a inicios de la pandemia en el 2020 y las últimas del persente año 2021.

In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
csv_files = glob.glob('./2020/*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

['./2020\\HUGO LOPEZ-GATELL RAMIREZ1.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ10.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ11.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ12.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ13.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ14.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ15.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ16.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ17.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ18.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ19.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ2.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ20.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ21.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ22.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ3.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ4.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ5.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ6.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ7.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ8.csv', './2020\\HUGO LOPEZ-GATELL RAMIREZ9.csv']


In [3]:
list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de 
# archivo a través de globbing y 
# el resultado final será la lista dataframes
for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola
list_data
 
pd.concat(list_data,ignore_index=True)['Texto']

0                     Muchas gracias, doctor Reyes Terán.
1       Ahora le pido al doctor José Luis Alomía si no...
2                                 Gracias, doctor Alomía.
3       Abrimos para preguntas. Por favor, compañera, ...
4        Le contesto la segunda y le paso a su compatr...
                              ...                        
2049    No hay una previsión anticipada de cierre de a...
2050    Y aquí ratifico la instrucción del señor presi...
2051                                             Gracias.
2052                                                  ---
2053                                              VE 823.
Name: Texto, Length: 2054, dtype: object

In [4]:
informes20 = pd.concat(list_data,ignore_index=True)['Texto']
informes20

0                     Muchas gracias, doctor Reyes Terán.
1       Ahora le pido al doctor José Luis Alomía si no...
2                                 Gracias, doctor Alomía.
3       Abrimos para preguntas. Por favor, compañera, ...
4        Le contesto la segunda y le paso a su compatr...
                              ...                        
2049    No hay una previsión anticipada de cierre de a...
2050    Y aquí ratifico la instrucción del señor presi...
2051                                             Gracias.
2052                                                  ---
2053                                              VE 823.
Name: Texto, Length: 2054, dtype: object

In [5]:
informe20 = np.array(informes20)
informe20

array([' Muchas gracias, doctor Reyes Terán.',
       'Ahora le pido al doctor José Luis Alomía si nos resume la situación actual en México.',
       ' Gracias, doctor Alomía.', ..., 'Gracias.', '---', 'VE 823.'],
      dtype=object)

In [6]:
import random

informe20[random.randint(0, 2053)]

'Gracias, ahí está el teléfono: 800 00 44 800, es la Unidad de Inteligencia Epidemiológica y Sanitaria de la Secretaría de Salud.'

## 2. Procesaiento de texto

Ahora es necesario normalizar el texto con fin de utilizarlo en modelado de tópicos.

El texto lo vamos a tratar con al menos los siguientes requisitos:

1. Utilizar todas las palabras en minúsculas
2. Eliminar los signos de puntuación
3. Eliminar palabras vacias 
    
Realiza esta normalización utilizando el módulo *spacy* con el modelo `es_core_news_sm`, pero sientete con la libertad de cargar el modelo 
`es_core_news_md` o inclusive el modelo `es_core_news_lg`. 

El corpus procesado (*normalizado* también se le dice) se guardará en una lista de textos que llamaremos `informe20_tratado`

In [7]:
import spacy
nlp = spacy.load("es_core_news_sm")

In [8]:
def procesa_informe(doc): 
    
    return [
        token.norm_ for token in doc
        if token.is_alpha and not token.like_num and not token.is_stop and
           token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']
    ]

informe20_tratado = [procesa_informe(doc) for doc in nlp.pipe(informes20)]

Y utiliza esta celda para probar si se normalizaron vien las participaciones realizadas.

In [9]:
i = random.randint(0, (len(informe20_tratado) - 1))
print(i)
print(informes20[i])
print("********************")
print(informe20_tratado[i])

2007
Esto no es algo nuevo, esto es la extensión de la medida que tomaron ya desde que estaba la epidemia sólo en China y que entró en vigor el domingo 2 de febrero a las 5:00 de la tarde y que básicamente consiste en que connacionales que venían de, en ese caso China, ahora será Europa, no podían estar en Estados Unidos.
********************
['extensión', 'medida', 'tomaron', 'epidemia', 'china', 'entró', 'vigor', 'domingo', 'febrero', 'consiste', 'connacionales', 'venían', 'caso', 'china', 'europa', 'unidos']


Ya realizado el tratamiento de las participaciones nos quedaremos con aquellas que cuenten con más de 3 palabras normalizadas.

In [10]:
np_lists = np.array(informe20_tratado)

filt = []
for i in range(0,len(informe20_tratado)):
    filt_i = len(np_lists[i]) >= 4
    filt.append(filt_i)
informe20_tratado = list(np_lists[filt])
print("\n", informe20_tratado)


 [['gracias', 'doctor', 'reyes', 'terán'], ['pido', 'doctor', 'josé', 'luis', 'alomía', 'resume', 'situación', 'actual', 'méxico'], ['abrimos', 'preguntas', 'favor', 'compañera', 'hernández', 'favor', 'compañero', 'hernández', 'ruth', 'favor'], ['contesto', 'paso', 'compatriota', 'sonorense', 'respuesta'], ['pregunten', 'muro', 'puestos', 'inspección', 'cerrar', 'carreteras', 'fronteras', 'respuesta', 'ahorita', 'tengamos', 'transmisión', 'mundo', 'recomienda', 'impedir', 'tránsito', 'personas', 'comercio', 'nacional', 'internacional'], ['sirve', 'sirve', 'detener', 'avance', 'epidemia', 'epidemia', 'virus', 'trasmite', 'vía', 'respiratoria', 'rápido', 'virus', 'transmitir', 'aparezcan', 'síntomas', 'filtro', 'situación', 'restricción', 'movilidad', 'enfocamos', 'persona', 'síntomas', 'pasando', 'personas', 'infectadas', 'síntomas'], ['respuesta', 'modelo', 'llama', 'mitigación', 'reducir', 'velocidad', 'transmisión', 'detectando', 'casos', 'identificando', 'contactos', 'atendiéndolos

<ipython-input-10-25125ef26f4f>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np_lists = np.array(informe20_tratado)


## 3. Modelado de tópicos con LDA

Ahora, ya con el corpus tratado, desarrolla tu modelo LDA en *gensim*, siguiendo los siguientes pasos:

1. Crea un diccionario con el informe tratado, donde elimines a todas las
   palabras que no aparezcan en al menos `min_df` documentos, y las palabras que 
   aparezcan el más del `max_df` porciento de documentos (por default `min_df = 5`
   y `max_df = 0.3`).
2. Genera un corpus listo para su uso, aplicando el metodo de bolsa de palabras.
3. Aplica el método para modelar con LDA. Establece el número de iteraciones para
   el método de estimación así como el numero de pasos que realiza el algoritmo
   de optimización iterativa (por default 100 y 5, respectivamente).
   
Guarda el modelo en la variable `modelo_lda` (opcionalmente puedes guardar el modelo si lo quieres usar más adelante)


In [11]:
import warnings
warnings.filterwarnings('ignore')

from gensim import models, corpora

# Genera el diccionario de palabras
diccionario20 = corpora.Dictionary(informe20_tratado)
diccionario20.filter_extremes(no_below=5, no_above=0.3)
 
# Extrae las características en forma de BOW
corpus20 = [diccionario20.doc2bow(doc) for doc in informe20_tratado]

Y ahora hacemos el modelado de tópicos con LDA

In [12]:
# Número de tópicos
n_topicos = 15

# Genera el modelo LDA
modelo_lda_20 = models.LdaModel(corpus=corpus20, num_topics=n_topicos, id2word=diccionario20, iterations=100, passes=10)

# Si quieres guardar el modelo se hace esto
#modelo_lda.save("sonetosLDA.model")

Y podemos usar otros métodos como el de *Hierachical Dirichlet Process*

In [13]:
modelo_hdp_20 = models.hdpmodel.HdpModel(corpus20, diccionario20, T=n_topicos)

Y ahora veamos los modelos como se definen por sus primeras 6 palabras clave 

In [14]:
for (i, topico) in modelo_lda_20.print_topics(num_topics=n_topicos, num_words=6):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.020*"población" + 0.019*"compañera" + 0.017*"méxico" + 0.017*"país" + 0.013*"personas" + 0.011*"vacaciones"
----------topico 1--------------------
0.034*"caso" + 0.028*"personas" + 0.022*"virus" + 0.020*"salud" + 0.017*"sistema" + 0.016*"coronavirus"
----------topico 2--------------------
0.027*"salud" + 0.021*"pregunta" + 0.017*"organización" + 0.017*"medidas" + 0.016*"favor" + 0.014*"doctor"
----------topico 3--------------------
0.029*"caso" + 0.028*"consejo" + 0.022*"casos" + 0.017*"salubridad" + 0.015*"casa" + 0.014*"autoridad"
----------topico 4--------------------
0.102*"casos" + 0.028*"salud" + 0.019*"epidemia" + 0.015*"méxico" + 0.012*"transmisión" + 0.012*"nacional"
----------topico 5--------------------
0.029*"situación" + 0.024*"internacional" + 0.021*"escenario" + 0.019*"salud" + 0.016*"países" + 0.013*"reglamento"
----------topico 6--------------------
0.037*"casos" + 0.028*"transmisión" + 0.025*"personas" + 0.025*"persona" + 0.019

In [15]:
for (i, topico) in modelo_hdp_20.print_topics(num_words=5):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.006*circunstancias + 0.006*social + 0.006*boca + 0.006*hernández + 0.006*fases
----------topico 1--------------------
0.005*incluye + 0.005*centros + 0.005*edad + 0.005*gobernador + 0.005*mantenido
----------topico 2--------------------
0.008*relevante + 0.006*característica + 0.006*centinela + 0.005*patrón + 0.005*llamadas
----------topico 3--------------------
0.006*respuestas + 0.006*significado + 0.005*tratar + 0.005*cofepris + 0.005*productos
----------topico 4--------------------
0.007*marzo + 0.006*empresa + 0.006*negativos + 0.005*fundamentales + 0.004*jabón
----------topico 5--------------------
0.006*señor + 0.006*ley + 0.005*volver + 0.005*responder + 0.005*seguir
----------topico 6--------------------
0.006*duda + 0.005*tomar + 0.005*lineamientos + 0.004*tuvieran + 0.004*venga
----------topico 7--------------------
0.005*normal + 0.005*hacienda + 0.005*indispensable + 0.004*importantísimo + 0.004*agradecimiento
----------topico 8----

Y ahora veamos como se clasifica un documento utilizando tanto LDA como HDP

In [16]:
i = random.randint(0, (len(informe20) - 1))

respuesta20 = informes20[i]
ids_20 = diccionario20.doc2bow(informe20_tratado[i])

print("Informe: \n\n" + respuesta20)

topicos_lda_20 = modelo_lda_20[ids_20]
topicos_lda_20.sort(key=lambda x:x[1], reverse=True)

topicos_hdp_20 = modelo_hdp_20[ids_20]
topicos_hdp_20.sort(key=lambda x:x[1], reverse=True)

print()
print("Pertenece a los tópicos (con el modelo LDA)")
for (topico, peso) in topicos_lda_20:
    print("\tTopico {} con un peso de {}".format(topico, peso))

print("Pertenece a los tópicos (con el modelo HDP)")
for (topico, peso) in topicos_hdp_20:
    print("\tTopico {} con un peso de {}".format(topico, peso))

Informe: 

Y es el momento en donde es más conveniente adelantar las intervenciones que se puede ejecutar para tener un alcance masivo. El alcance masivo de las intervenciones se logra con las intervenciones de mitigación, de mitigación comunitaria y es precisamente el distanciamiento social.

Pertenece a los tópicos (con el modelo LDA)
	Topico 12 con un peso de 0.7067030668258667
	Topico 7 con un peso de 0.20662038028240204
Pertenece a los tópicos (con el modelo HDP)
	Topico 0 con un peso de 0.7275101477876799
	Topico 13 con un peso de 0.18559025463382098


Tambien se puede obtener la lista de palabras que describen a cada tópico.

Mejor aún, utiliza *pyLDAvis* para visualizar y analizar los tópicos desarrollados.

In [17]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(modelo_lda_20, corpus20, diccionario20)
pyLDAvis.display(vis_data)

In [18]:
vis_data = gensimvis.prepare(modelo_hdp_20, corpus20, diccionario20)
pyLDAvis.display(vis_data)

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 4 Evaluando modelos

Para evaluar un modelo, la forma más sencilla es utilizando el modelo para evaluar la perplejidad del corpus

In [19]:
print(f"El logaritomo de la perplejidad del corpus al modelo LDA es {modelo_lda_20.log_perplexity(corpus20)}")

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


El logaritomo de la perplejidad del corpus al modelo LDA es -7.05479776412488


Otro método es analizando la [coherencia de los tópicos](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf), ponemos algunos resultados:

In [20]:
cm_lda_20 = models.CoherenceModel(model=modelo_lda_20, texts=informe20_tratado, coherence='c_v')
cm_hdp_20 = models.CoherenceModel(model=modelo_hdp_20, texts=informe20_tratado, coherence='c_v')


print("Coherencia del modelo LDA (C_V): {}".format(cm_lda_20.get_coherence()))
for (topico, coherencia) in enumerate(cm_lda_20.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

print("\n\nCoherencia del modelo HDP (C_V): {}".format(cm_hdp_20.get_coherence()))
for (topico, coherencia) in enumerate(cm_hdp_20.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherencia del modelo LDA (C_V): 0.3450535159185491
	Tópico 0, coherencia 0.520736403625742
	Tópico 1, coherencia 0.24859291534189767
	Tópico 2, coherencia 0.42023435872798387
	Tópico 3, coherencia 0.25782022380782726
	Tópico 4, coherencia 0.2633807613986129
	Tópico 5, coherencia 0.39993463280397357
	Tópico 6, coherencia 0.417904798229921
	Tópico 7, coherencia 0.34249007177296775
	Tópico 8, coherencia 0.34961034285189574
	Tópico 9, coherencia 0.3097413894542439
	Tópico 10, coherencia 0.34137512151930455
	Tópico 11, coherencia 0.4900190595903472
	Tópico 12, coherencia 0.3392109679123332
	Tópico 13, coherencia 0.2055105388158104
	Tópico 14, coherencia 0.2692411529253761


Coherencia del modelo HDP (C_V): 0.7371866296365162
	Tópico 0, coherencia 0.6753902447775927
	Tópico 1, coherencia 0.7714940745596585
	Tópico 2, coherencia 0.7264697749341524
	Tópico 3, coherencia 0.7602533321337535
	Tópico 4, coherencia 0.7249884605362241
	Tópico 5, coherencia 0.6804084968820191
	Tópico 6, coherencia 0

Con la coherencia se puede obtener el conjunto de palabras que mejor definen cada tópico

In [21]:
print("Las palabra que definen los tópicos en LDA\n")
t_palabras = cm_lda_20.top_topics_as_word_lists(model=modelo_lda_20, dictionary=diccionario20)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")
    
print("\n\nLas palabra que definen los tópicos en HDP\n")
t_palabras = cm_hdp_20.top_topics_as_word_lists(model=modelo_hdp_20, dictionary=diccionario20)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")

Las palabra que definen los tópicos en LDA

0 - población, compañera, méxico, país, personas, vacaciones, vive, zona, condiciones, ángeles, actividades, mayores, palabra, resultado, epidemia, araceli, blanca, guía, compañero, filtro
1 - caso, personas, virus, salud, sistema, coronavirus, riesgo, actividades, enfermedades, número, persona, sociedad, tipo, infección, pública, grande, conjunto, medida, infecciones, respuesta
2 - salud, pregunta, organización, medidas, favor, doctor, mundial, emergencia, pido, gusto, escenario, preparación, minutos, ahorita, respuesta, recomendaciones, quedan, méxico, natalia, alomía
3 - caso, consejo, casos, salubridad, casa, autoridad, quédate, sesión, méxico, sanitaria, fase, salud, medida, virus, empieza, persona, síntomas, secretaría, intervenciones, escenario
4 - casos, salud, epidemia, méxico, transmisión, nacional, cantidad, curva, personas, caso, número, respuesta, semana, pública, contactos, semanas, atención, enfermedad, comité, ahorita
5 - situ

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##### Realizamos los mismos procedimientos para el análisis de tópicos del 2021

In [22]:
# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
csv_files = glob.glob('./2021/*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['./2021\\HUGO LOPEZ-GATELL RAMIREZ1.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ10.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ11.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ12.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ13.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ14.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ15.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ16.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ17.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ18.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ19.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ2.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ20.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ21.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ22.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ3.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ4.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ5.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ6.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ7.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ8.csv', './2021\\HUGO LOPEZ-GATELL RAMIREZ9.csv']


In [23]:
list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y 
# el resultado final será la lista dataframes
for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)
#Para chequear que todo está bien, mostramos la list_data por consola
list_data
 
pd.concat(list_data,ignore_index=True)['Texto']

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0                    Se escucha muy bien, Juan. Adelante.
1       Excelente. Muchas gracias, gracias a todas, bu...
2       Antes de hacerles un par de preguntas que quis...
3       Por eso me parece muy acertado, como lo formul...
4       Y dado que somos una república federal, tambié...
                              ...                        
2320    Y sobre lo que me preguntaba, ya me pasaron la...
2321    Le invitamos a usted, si tiene 50 años de edad...
2322    Nuestro interés, el interés del Gobierno de Mé...
2323    Muchas gracias. Nos vemos mañana. Gracias, doc...
2324                                                  ---
Name: Texto, Length: 2325, dtype: object

In [24]:
informes21 = pd.concat(list_data,ignore_index=True)['Texto']
informes21

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0                    Se escucha muy bien, Juan. Adelante.
1       Excelente. Muchas gracias, gracias a todas, bu...
2       Antes de hacerles un par de preguntas que quis...
3       Por eso me parece muy acertado, como lo formul...
4       Y dado que somos una república federal, tambié...
                              ...                        
2320    Y sobre lo que me preguntaba, ya me pasaron la...
2321    Le invitamos a usted, si tiene 50 años de edad...
2322    Nuestro interés, el interés del Gobierno de Mé...
2323    Muchas gracias. Nos vemos mañana. Gracias, doc...
2324                                                  ---
Name: Texto, Length: 2325, dtype: object

In [25]:
informe21 = np.array(informes21)
informe21

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Se escucha muy bien, Juan. Adelante.',
       'Excelente. Muchas gracias, gracias a todas, bueno a la doctora\xa0Giorguli\xa0y a todos ustedes.',
       'Antes de hacerles un par de preguntas que quisiéramos hacerles a nuestros ponentes, una por mi parte, otra por parte del doctor López Ridaura, retomo sobre lo que ya fue explicado para destacar lo que para nosotros es una convicción: la importancia de escuchar a distintas personalidades en la sociedad, en el mundo académico y científico, por ejemplo, representado por estas 14 instituciones, tanto nacionales, como incluso internacionales, tales como la Comisión Económica para América Latina de Naciones Unidas o el Banco Mundial, y también escuchar a la sociedad, la sociedad civil organizada nos ha aportado también elementos valiosísimos, en algunos casos de asesoría, de análisis, de reflexión, de crítica y también en muchos otros casos de soluciones prácticas, se ha sumado ampliamente.',
       ...,
       'Nuestro interés, el 

In [26]:
informe21[random.randint(0, (len(informe21) - 1))]

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'Lo dijimos esta mañana. Unas cuantas más, son las extras, las que están en el material extra, es esta mera, exactamente.'

In [27]:
def procesa_informe(doc): 
    
    return [
        token.norm_ for token in doc
        if token.is_alpha and not token.like_num and not token.is_stop and
           token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']
    ]

informe21_tratado = [procesa_informe(doc) for doc in nlp.pipe(informes21)]

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
np_lists = np.array(informe21_tratado)

filt = []
for i in range(0,len(informe21_tratado)):
    filt_i = len(np_lists[i]) >= 6
    filt.append(filt_i)
informe21_tratado = list(np_lists[filt])
print("\n", informe21_tratado)


 [['hacerles', 'par', 'preguntas', 'quisiéramos', 'hacerles', 'ponentes', 'doctor', 'lópez', 'ridaura', 'retomo', 'explicado', 'destacar', 'convicción', 'importancia', 'escuchar', 'personalidades', 'sociedad', 'mundo', 'académico', 'científico', 'representado', 'instituciones', 'nacionales', 'internacionales', 'comisión', 'económica', 'américa', 'latina', 'naciones', 'unidas', 'banco', 'mundial', 'escuchar', 'sociedad', 'sociedad', 'civil', 'organizada', 'aportado', 'elementos', 'valiosísimos', 'casos', 'asesoría', 'análisis', 'reflexión', 'crítica', 'casos', 'soluciones', 'prácticas', 'sumado'], ['acertado', 'formula', 'doctor', 'rivera', 'asesoría', 'mexicano', 'gobierno', 'dije', 'inicio', 'dije', 'reducida', 'realidad', 'asesoría', 'mexicano'], ['república', 'federal', 'serie', 'recomendaciones', 'relevancia', 'poderes', 'locales', 'olvidemos', 'entidades', 'federativas', 'gobiernos', 'responsabilidad', 'específica', 'autoridades', 'sanitarias'], ['diversidad', 'respuesta', 'fenóm

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
i = random.randint(0, (len(informe21_tratado) - 1))
print(i)
print(informes21[i])
print("********************")
print(informe21_tratado[i])

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1468
Óscar, por favor, adelante.
********************
['resto', 'número', 'grande', 'recordar', 'utilizando', 'vacuna', 'cansino', 'dosis', 'esquema', 'completo']


In [30]:
import warnings
warnings.filterwarnings('ignore')

from gensim import models, corpora

# Genera el diccionario de palabras
diccionario21 = corpora.Dictionary(informe21_tratado)
diccionario21.filter_extremes(no_below=5, no_above=0.3)
 
# Extrae las características en forma de BOW
corpus21 = [diccionario21.doc2bow(doc) for doc in informe21_tratado]

C:\Users\Daniel Fernandez\anaconda3\envs\spacy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# Número de tópicos
n_topicos = 15

# Genera el modelo LDA
modelo_lda_21 = models.LdaModel(corpus=corpus21, num_topics=n_topicos, id2word=diccionario21, iterations=100, passes=10)

# Si quieres guardar el modelo se hace esto
#modelo_lda.save("sonetosLDA.model")

In [32]:
modelo_hdp_21 = models.hdpmodel.HdpModel(corpus21, diccionario21, T=n_topicos)

In [33]:
for (i, topico) in modelo_lda_21.print_topics(num_topics=n_topicos, num_words=6):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.035*"inmunidad" + 0.022*"población" + 0.021*"personas" + 0.013*"cantidad" + 0.012*"años" + 0.011*"méxico"
----------topico 1--------------------
0.032*"doctor" + 0.032*"epidemia" + 0.023*"salud" + 0.017*"reducción" + 0.016*"preguntas" + 0.012*"semanas"
----------topico 2--------------------
0.035*"cofepris" + 0.035*"proceso" + 0.025*"países" + 0.020*"vacuna" + 0.018*"méxico" + 0.014*"sanitaria"
----------topico 3--------------------
0.039*"vacunación" + 0.034*"población" + 0.027*"personas" + 0.022*"vacunas" + 0.017*"covid" + 0.015*"eventos"
----------topico 4--------------------
0.042*"salud" + 0.020*"entidad" + 0.018*"semana" + 0.017*"doctor" + 0.015*"federativa" + 0.012*"méxico"
----------topico 5--------------------
0.041*"personas" + 0.030*"vacunas" + 0.029*"programa" + 0.025*"vacunación" + 0.024*"méxico" + 0.019*"gobierno"
----------topico 6--------------------
0.043*"personas" + 0.031*"vacunación" + 0.023*"salud" + 0.018*"persona" + 0.018*

In [34]:
for (i, topico) in modelo_hdp_21.print_topics(num_words=5):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

----------topico 0--------------------
0.006*campeche + 0.005*cerrados + 0.005*semana + 0.005*próxima + 0.005*serie
----------topico 1--------------------
0.008*leves + 0.006*lineamiento + 0.005*resultados + 0.005*toca + 0.004*persona
----------topico 2--------------------
0.005*relato + 0.005*completo + 0.005*recuerdo + 0.004*elementos + 0.004*reciente
----------topico 3--------------------
0.006*análisis + 0.005*colegas + 0.004*nacional + 0.004*calibración + 0.004*duración
----------topico 4--------------------
0.007*vacunar + 0.005*benéfico + 0.005*baja + 0.005*argentina + 0.005*términos
----------topico 5--------------------
0.006*luz + 0.005*acumulada + 0.005*ridaura + 0.005*responsabilidad + 0.004*llamamos
----------topico 6--------------------
0.005*ejemplos + 0.005*vacunado + 0.004*correspondientes + 0.004*etapas + 0.004*contemplado
----------topico 7--------------------
0.006*julio + 0.005*completa + 0.005*zenteno + 0.005*seguridad + 0.004*tiempos
----------topico 8-----------

In [37]:
i = random.randint(0, (len(informe21) - 1))

respuesta21 = informes21[i]
ids_21 = diccionario21.doc2bow(informe21_tratado[i])

print("Informe: \n\n" + respuesta21)

topicos_lda_21 = modelo_lda_21[ids_21]
topicos_lda_21.sort(key=lambda x:x[1], reverse=True)

topicos_hdp_21 = modelo_hdp_21[ids_21]
topicos_hdp_21.sort(key=lambda x:x[1], reverse=True)

print()
print("Pertenece a los tópicos (con el modelo LDA)")
for (topico, peso) in topicos_lda_21:
    print("\tTopico {} con un peso de {}".format(topico, peso))

print("Pertenece a los tópicos (con el modelo HDP)")
for (topico, peso) in topicos_hdp_21:
    print("\tTopico {} con un peso de {}".format(topico, peso))

Informe: 

Recomendamos ampliamente que se vacunen y sigue abierta esa posibilidad. Si usted tiene 60 años de edad o más o conoce a una persona en esta situación, recomiéndele o tome usted esta recomendación, acérquense a vacunar en su localidad cuando llegue la vacuna contra el COVID.

Pertenece a los tópicos (con el modelo LDA)
	Topico 13 con un peso de 0.44871005415916443
	Topico 8 con un peso de 0.4429495930671692
Pertenece a los tópicos (con el modelo HDP)
	Topico 1 con un peso de 0.4730475782471732
	Topico 12 con un peso de 0.41859324364748746


In [38]:
vis_data = gensimvis.prepare(modelo_lda_21, corpus21, diccionario21)
pyLDAvis.display(vis_data)

In [39]:
vis_data = gensimvis.prepare(modelo_hdp_21, corpus21, diccionario21)
pyLDAvis.display(vis_data)

In [40]:
print(f"El logaritomo de la perplejidad del corpus al modelo LDA es {modelo_lda_21.log_perplexity(corpus21)}")

El logaritomo de la perplejidad del corpus al modelo LDA es -6.942941214980482


In [41]:
cm_lda_21 = models.CoherenceModel(model=modelo_lda_21, texts=informe21_tratado, coherence='c_v')
cm_hdp_21 = models.CoherenceModel(model=modelo_hdp_21, texts=informe21_tratado, coherence='c_v')


print("Coherencia del modelo LDA (C_V): {}".format(cm_lda_21.get_coherence()))
for (topico, coherencia) in enumerate(cm_lda_21.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

print("\n\nCoherencia del modelo HDP (C_V): {}".format(cm_hdp_21.get_coherence()))
for (topico, coherencia) in enumerate(cm_hdp_21.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

Coherencia del modelo LDA (C_V): 0.3360457857392859
	Tópico 0, coherencia 0.3280752190629656
	Tópico 1, coherencia 0.30081137912518285
	Tópico 2, coherencia 0.3664372515448019
	Tópico 3, coherencia 0.24123019352200487
	Tópico 4, coherencia 0.30694655716934643
	Tópico 5, coherencia 0.23622584243325212
	Tópico 6, coherencia 0.3478850044703785
	Tópico 7, coherencia 0.543340067369454
	Tópico 8, coherencia 0.269049484961933
	Tópico 9, coherencia 0.2777735986543589
	Tópico 10, coherencia 0.4068782270325645
	Tópico 11, coherencia 0.2822090763710509
	Tópico 12, coherencia 0.42022737081485867
	Tópico 13, coherencia 0.23266957923701806
	Tópico 14, coherencia 0.4809279343201182


Coherencia del modelo HDP (C_V): 0.7480246417784324
	Tópico 0, coherencia 0.778724619185273
	Tópico 1, coherencia 0.7475645963529892
	Tópico 2, coherencia 0.7793866488233869
	Tópico 3, coherencia 0.7422886420653583
	Tópico 4, coherencia 0.7012829464021899
	Tópico 5, coherencia 0.7616144930012665
	Tópico 6, coherencia 0.7

In [42]:
print("Las palabra que definen los tópicos en LDA\n")
t_palabras = cm_lda_21.top_topics_as_word_lists(model=modelo_lda_21, dictionary=diccionario21)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")
    
print("\n\nLas palabra que definen los tópicos en HDP\n")
t_palabras = cm_hdp_21.top_topics_as_word_lists(model=modelo_hdp_21, dictionary=diccionario21)
for (topico, palabras) in enumerate(t_palabras):
    print(f"{topico} - {', '.join(palabras)}")

Las palabra que definen los tópicos en LDA

0 - inmunidad, población, personas, cantidad, años, méxico, caso, duración, vacunas, datos, india, sarampión, epidemia, protección, cubrebocas, porcentaje, meses, mundo, tengamos, lograr
1 - doctor, epidemia, salud, reducción, preguntas, semanas, lópez, epidémica, méxico, empezó, abrir, secretario, semana, actividad, vacunación, riesgo, juan, favor, baja, vacunas
2 - cofepris, proceso, países, vacuna, méxico, sanitaria, epidemia, europa, mundo, autorización, riesgo, digo, lotes, liomont, gobierno, liberación, protocolos, electoral, gusto, nivel
3 - vacunación, población, personas, vacunas, covid, eventos, vacuna, grupo, propósito, programa, vacunado, diciembre, asociados, méxico, empieza, virus, leves, reacciones, esavis, vacunada
4 - salud, entidad, semana, doctor, federativa, méxico, nacional, semáforo, número, conferencia, organización, gracias, información, verde, amarillo, pública, públicos, vacunación, entidades, atención
5 - personas, 